# Notebook con algoritmos utilizados en el taller

---
## Punto 1


---
## Punto 2


Considere el cifrado de Hill sobre $\mathbb{Z}_{26}$ con bloques de tamaño 2, definido por: 

$$E_{k}(x) = xK (mód 26)$$

donde $x \in \mathbb{Z}_{26}^{2}$ es un vector fila y $K \in \mathbb{Z}_{26}^{2x2}$ es la matriz clave.
Sea la matriz 

$$ 
K = 
\begin{bmatrix} 
24 & 5 \\ 
13 & 7
 \end{bmatrix}
$$

---
a. Determine si la matriz K constituye una clave válida para el cifrado de Hill. Justifique su respuesta.

Sea el alfabeto L con |L| = n. Una matriz $ K \in \mathbb{Z}_{n}^{i x j} $  constituye una clave válida para el cifrado de Hill si:
- i = j. Es decir, una matriz cuadrada
- K invertible en $\mathbb{Z}_{n}$

Para que K sea invertible en $\mathbb{Z}_{n}$ se debe de cumplir que gcd(det(K) mód n, n) = 1. Es decir, que el determinante de la matriz bajo el módulo del tamaño del alfabeto sea coprimo con el tamaño del alfabeto.

Por lo tanto, para nuestro caso:
- K es cuadrada
- n = 26

In [106]:
import numpy as np

In [107]:
def euclides_algorithm(a, b):
    if a < b:
        a, b = b, a
    if b == 0:
        return a
    else:
        return euclides_algorithm(b, a % b)

In [108]:
def id_valid_hill_key(key: np.array, n: int) -> bool:
    is_valid = False
    if key.shape[0] == key.shape[1]:
        det_k = np.linalg.det(key)
        if det_k != 0 and det_k % n != 0:
            if euclides_algorithm(int(det_k), n) == 1:
                is_valid = True
    return is_valid

In [109]:
K = np.array([
    [24, 5],
    [13, 7]
])

In [110]:
print("K es una matriz válida" if id_valid_hill_key(K, 26) else "K no es una matriz válida")

K es una matriz válida


Dado que se cumplieron todas las condiciones, la matriz K si es una clave válida para el cifrado de Hill.

---
b. Se interceptaron los siguientes textos cifrados, que fueron generados utilizando la matriz K. Descífrelos:
<center>
c = [M, L], 
c' = [R, E]
<center>


Para descifrarlos requerimos encontrar $K^{-1}$

In [111]:
def extended_euclidean(a, b):
    if a == 0:
        return b, 0, 1
    gcd, t1, s1 = extended_euclidean(b % a, a)
    t = s1 - (b // a) * t1
    s = t1    
    return gcd, t, s

In [112]:
def invert_key(key: np.array, n: int) -> np.array:
    det_k = int(np.linalg.det(key))
    gcd, det_inv, s = extended_euclidean(det_k % n, n)
    det_inv = det_inv % n
    key_inv = det_inv * np.array([
        [key[1][1], -key[0][1]], 
        [-key[1][0], key[0][0]]])
    return key_inv % n

In [ ]:
print(invert_key(K, 26))

[[19  5]
 [13  2]]


Ahora procedemos a construir la función que nos permitirá descifrar el mensaje cifrado.

In [114]:
def decrypt_hill(cipher_v: np.array, key: np.array, n: int) -> str:
    alphabet = {chr(i + ord('A')): i for i in range(26)}
    cipher_v = np.array([alphabet[char] for char in cipher_v])
    key_inv = invert_key(key, n)
    decrypted_v = np.dot(cipher_v, key_inv) % n
    decrypted_v = np.array([chr(int(num) + ord('A')) for num in decrypted_v])
    return decrypted_v
    

In [116]:
c = np.array(['M', 'L'])
c_p = np.array(['R', 'E'])
decrypted_message = decrypt_hill(c, K, 26)
decrypted_message_p = decrypt_hill(c_p, K, 26)
print("Mensaje descifrado c:", decrypted_message)
print("Mensaje descifrado c':", decrypted_message_p)

Mensaje descifrado c: ['H' 'E']
Mensaje descifrado c': ['L' 'P']


---
c. Suponga ahora un escenario de ataque de texto claro conocido, en el cual el atacante dispone
de pares $(m_{i},c_{i})$ (plaintext, ciphertext), pero desconoce la matriz K, donde $c_{i} = m_{i}K$ (mód 26)

- Explique cómo modelaría matemáticamente el problema de recuperar la matriz K a
partir de dicha información.

Dado que deseamos recuperar la matriz, tomemos la forma $c_{i} = m_{i}K$ (mód 26), por lo tanto, 
si se expande se tiene que: 

$$ 
\begin{bmatrix} 
c_{i}^{0} & c_{i}^{1} & ... & c_{i}^{n}
 \end{bmatrix}

= 

\begin{bmatrix} 
m_{i}^{0} & m_{i}^{1} & ... & m_{i}^{n}
 \end{bmatrix}

\begin{bmatrix} 
k_{0, 0} & k_{0, 1} & ... & k_{0, n} \\
k_{1, 0} & k_{1, 1} & ... & k_{1, n} \\
... & ... & ... & ... \\
k_{n, 0} & k_{n, 1} & ... & k_{n, n}
 \end{bmatrix}
$$

Y por lo tanto lo podemos ver como un sistema de ecuaciones: 


$$
\begin{cases}
m_{i}^{0} \cdot k_{0,0} + m_{i}^{1} \cdot k_{1, 0} + ... + m_{i}^{n} \cdot k_{0,n} = c_{i}^{0} \\
m_{i}^{0} \cdot k_{0,1} + m_{i}^{1} \cdot k_{1, 1} + ... + m_{i}^{n} \cdot k_{1,n} = c_{i}^{1} \\
... \\
m_{i}^{0} \cdot k_{0,n} + m_{i}^{1} \cdot k_{1, n} + ... + m_{i}^{n} \cdot k_{n,n} = c_{i}^{n}
\end{cases}
$$

Dado que cada fila se expresa como una ecuación independiente respecto a las variables $k_{l, n}$, requerimos de varios pares $(m_{i}, c_{i})$ para poder obtener las suficientes ecuaciones para formar el sistema de las variables $k_{l, n}$, teniendo en cuenta que $l$ representa una fila de variables dentro de la matriz K.

- Explique cómo modelaría matemáticamente el problema de recuperar la matriz K a
partir de dicha información.

**Modelado matemático del problema:**

Dado que conocemos pares $(m_i, c_i)$ y la relación $c_i = m_i K \pmod{26}$, podemos modelar el problema de la siguiente manera:

**1. Expandiendo la ecuación matricial:**

Para una matriz K de tamaño $n \times n$, la ecuación $c_i = m_i K$ se puede expandir como:

$$ 
\begin{bmatrix} 
c_{i}^{0} & c_{i}^{1} & \cdots & c_{i}^{n-1}
\end{bmatrix}
= 
\begin{bmatrix} 
m_{i}^{0} & m_{i}^{1} & \cdots & m_{i}^{n-1}
\end{bmatrix}
\begin{bmatrix} 
k_{0,0} & k_{0,1} & \cdots & k_{0,n-1} \\
k_{1,0} & k_{1,1} & \cdots & k_{1,n-1} \\
\vdots & \vdots & \ddots & \vdots \\
k_{n-1,0} & k_{n-1,1} & \cdots & k_{n-1,n-1}
\end{bmatrix}
\pmod{26}
$$

**2. Sistema de ecuaciones lineales:**

Cada par $(m_i, c_i)$ genera $n$ ecuaciones lineales (una por cada componente del vector cifrado):

$$
\begin{cases}
m_{i}^{0} \cdot k_{0,0} + m_{i}^{1} \cdot k_{1,0} + \cdots + m_{i}^{n-1} \cdot k_{n-1,0} \equiv c_{i}^{0} \pmod{26} \\
m_{i}^{0} \cdot k_{0,1} + m_{i}^{1} \cdot k_{1,1} + \cdots + m_{i}^{n-1} \cdot k_{n-1,1} \equiv c_{i}^{1} \pmod{26} \\
\vdots \\
m_{i}^{0} \cdot k_{0,n-1} + m_{i}^{1} \cdot k_{1,n-1} + \cdots + m_{i}^{n-1} \cdot k_{n-1,n-1} \equiv c_{i}^{n-1} \pmod{26}
\end{cases}
$$

**3. Formulación matricial del sistema:**

Si recopilamos $n$ pares $(m_1, c_1), (m_2, c_2), \ldots, (m_n, c_n)$, podemos construir el sistema:

$$
\begin{bmatrix} 
m_1^{0} & m_1^{1} & \cdots & m_1^{n-1} \\
m_2^{0} & m_2^{1} & \cdots & m_2^{n-1} \\
\vdots & \vdots & \ddots & \vdots \\
m_n^{0} & m_n^{1} & \cdots & m_n^{n-1}
\end{bmatrix}
\begin{bmatrix} 
k_{0,0} & k_{0,1} & \cdots & k_{0,n-1} \\
k_{1,0} & k_{1,1} & \cdots & k_{1,n-1} \\
\vdots & \vdots & \ddots & \vdots \\
k_{n-1,0} & k_{n-1,1} & \cdots & k_{n-1,n-1}
\end{bmatrix}
\equiv
\begin{bmatrix} 
c_1^{0} & c_1^{1} & \cdots & c_1^{n-1} \\
c_2^{0} & c_2^{1} & \cdots & c_2^{n-1} \\
\vdots & \vdots & \ddots & \vdots \\
c_n^{0} & c_n^{1} & \cdots & c_n^{n-1}
\end{bmatrix}
\pmod{26}
$$

O de forma compacta: $MK \equiv C \pmod{26}$

**4. Solución del sistema:**

Si la matriz $M$ (formada por los vectores de texto plano) es invertible en $\mathbb{Z}_{26}$, podemos recuperar $K$ mediante:

$$K \equiv M^{-1}C \pmod{26}$$

El problema se reduce a resolver un sistema de ecuaciones lineales en aritmética modular, donde necesitamos suficientes pares $(m_i, c_i)$ para formar una matriz $M$ invertible y así recuperar la matriz clave $K$.

- Determine el número mínimo de pares $(m_{i},c_{i})$ necesarios para recuperar completamente
la matriz K e indique qué condición deben satisfacer dichos pares.


**Número mínimo de pares**

Para una matriz K de tamaño $n \times n$, tenemos $n^2$ incógnitas (elementos $k_{i,j}$).

Cada par $(m_i, c_i)$ proporciona n ecuaciones (una por cada columna de la matriz K), ya que:
- $c_i$ tiene n componentes
- $m_i$ tiene n componentes

Por lo tanto, el número mínimo de pares necesarios es:

$$\text{Número mínimo} = n$$

Con $n$ pares obtenemos $n \times n = n^2$ ecuaciones para $n^2$ incógnitas.

**Condición que deben satisfacer**

Los pares $(m_i, c_i)$ deben satisfacer que la matriz M formada por los vectores $m_i$ sea invertible (mód 26):

$$
M = \begin{bmatrix} 
m_1^0 & m_1^1 & \cdots & m_1^{n-1} \\
m_2^0 & m_2^1 & \cdots & m_2^{n-1} \\
\vdots & \vdots & \ddots & \vdots \\
m_n^0 & m_n^1 & \cdots & m_n^{n-1}
\end{bmatrix}
$$

**Condición:** $\det(M) \not\equiv 0 \pmod{26}$ y $\gcd(\det(M), 26) = 1$

Esto garantiza que los vectores $m_i$ sean linealmente independientes (módulo 26), permitiendo resolver el sistema:

$$C = MK \pmod{26}$$
$$K = M^{-1}C \pmod{26}$$

donde:
- $C$ es la matriz formada por los vectores $c_i$
- $M^{-1}$ es la inversa modular de M

---
## Punto 3

Hay un mensaje cifrado con el criptosistema de Vigenere:

In [ ]:
message = """
IAKEHSSFUGQMRWLWLREAFDWOYRBLVWXDYTQRYOSNYQJYRYIJSBVEJKSEYGPEEEYLGBATBIEDFOGHRUKJ
UALMBWLWLGPEBOHOIZINNGSJYQPEEVSEOPPTUDXKBRUAQHLWLNAMNOPJYQDEYYILBBWDVWWMCGMDUHVH
YENEPWPQUALFERQLBNBDNBSFYIMRLRRWWNTLRGLWLYQTGOIJYQZIQLRYBBWDBQIEIEVIAJLWLZWTUHVK
UVLCBPIDCGBLRUIVLVLIAJLGIQPEEHWSJVMCRRJUUXMAAGETIGBLRRJOCAMTNNILBRUTBBSMLTZAAGQG
NUMRFKIKMVKKNQHOYNSAAGXZCFEIYOHGBRZGBRHYIOMFBUIANTMTFWSGBBBAAGVWGRUBRUXGQNTKPDVW
ZHTLLGSFNFBRNBJJIZBHRSELBBZYBXQAAUBFNOPSHQJRRDOLBRJOGWPWUALWUHRQIHIRELZWXBVTSRVY
YGBOFDCYIBLMBURAHTJESRVWSBCPRHOSLBCNQKIJLBWMVOPTYPIRRIYDGBBHRUWSCQTIGWPWLRLRVGMF
AUWOQDRVMUMPERQAMRLWVWLSMZQLRWLWAEINQPSLBRZLVYIVXRMPVQXZYSWRRVXSVBCTUDPXUYMATXIX
LBUTUHZAFYIGRMYKNNALVWXDYEMDELHAHTPOBGIFNRZEQWLWQBWDFDAGFSIPCHEJYQWNGKIHUGPSUHHA
XABKARAOBNBAJLGCYQKRRDXMLRPEJDWSHQNEYWRGZRIRNWEDF
"""

In [4]:
message = message.replace("\n", "")
print(message)

IAKEHSSFUGQMRWLWLREAFDWOYRBLVWXDYTQRYOSNYQJYRYIJSBVEJKSEYGPEEEYLGBATBIEDFOGHRUKJUALMBWLWLGPEBOHOIZINNGSJYQPEEVSEOPPTUDXKBRUAQHLWLNAMNOPJYQDEYYILBBWDVWWMCGMDUHVHYENEPWPQUALFERQLBNBDNBSFYIMRLRRWWNTLRGLWLYQTGOIJYQZIQLRYBBWDBQIEIEVIAJLWLZWTUHVKUVLCBPIDCGBLRUIVLVLIAJLGIQPEEHWSJVMCRRJUUXMAAGETIGBLRRJOCAMTNNILBRUTBBSMLTZAAGQGNUMRFKIKMVKKNQHOYNSAAGXZCFEIYOHGBRZGBRHYIOMFBUIANTMTFWSGBBBAAGVWGRUBRUXGQNTKPDVWZHTLLGSFNFBRNBJJIZBHRSELBBZYBXQAAUBFNOPSHQJRRDOLBRJOGWPWUALWUHRQIHIRELZWXBVTSRVYYGBOFDCYIBLMBURAHTJESRVWSBCPRHOSLBCNQKIJLBWMVOPTYPIRRIYDGBBHRUWSCQTIGWPWLRLRVGMFAUWOQDRVMUMPERQAMRLWVWLSMZQLRWLWAEINQPSLBRZLVYIVXRMPVQXZYSWRRVXSVBCTUDPXUYMATXIXLBUTUHZAFYIGRMYKNNALVWXDYEMDELHAHTPOBGIFNRZEQWLWQBWDFDAGFSIPCHEJYQWNGKIHUGPSUHHAXABKARAOBNBAJLGCYQKRRDXMLRPEJDWSHQNEYWRGZRIRNWEDF


### Explicación del criptosistema de Vigenere

$$
C(X_i) = (X_i + K_i) \bmod 26
$$
Donde $X$ es el mensaje, $i$ es la posición y $K$ la clave

#### Ejemplo de cifrado:

$X$ = HOLA

$K$ = CLAVE

- Ambos debes quedar del mismo tamaño

$X$ = HOLA

$K$ = CLAV

- Pasamos la letra a su valor en numero

|Letra|Valor|
|---|---|
|H|7|
|O|14|
|L|11|
|A|0|
|C|2|
|L|11|
|A|0|
|V|21|

- Sumamos en módulo 26 y pasamos a letra

|Suma|Letra|
|---|---|
|9|J|
|25|Z|
|11|L|
|21|V|

Texto cifrado: JZLV



#### Funciones útiles

In [28]:
def letra_a_numero(letra):
    return ord(letra.upper()) - ord('A')

def numero_a_letra(numero):
    return chr(numero % 26 + ord('A'))

def cifrar_por_vigenere(mensaje:str, clave:str) -> str:
    # Tamaño de mensaje vs clave
    while len(mensaje) > len(clave):
        clave += clave
    
    while len(mensaje) != len(clave):
        clave = clave[:-1]
    
    # Pasar a letras
    mensaje_list = []
    for a in mensaje:
        mensaje_list.append(letra_a_numero(a))
        
    clave_list = []
    for a in clave:
        clave_list.append(letra_a_numero(a))
        
    # print(mensaje_list)
    # print(clave_list)
    
    #Sumar módulo 26
    cifrado_list = []
    for a in range(len(mensaje_list)):
        cifrado_list.append(
            mensaje_list[a] + clave_list[a]
        )
    
    for a in range(len(cifrado_list)):
        while not cifrado_list[a] < 26:
            cifrado_list[a]-=26
    
    # print(cifrado_list)
    
    # Pasar a texto
    out = ""
    for a in cifrado_list:
        out+=numero_a_letra(a)
        
    return out
    pass


### Ataque visto en clase
Intuición: La clave se repite todo el tiempo, entonces habrá un patrón cada cierto numero de letras si el mensaje tiene pedazos iguales

Se utiliza en indice de coincidencia (probabilidad de que 2 letras elejidas al azar sean iguales)
$$
IC = \frac{\sum_{i=0}^{25} f_i(f_i-1)}{N(N-1)}
$$
Donde $f_i$ es la frecuencia de la letra $i$ y $N$ es el total de letras del texto

- Las posiciones $0, k, 2k, 3k, \dots$ fueron cifradas con la misma letra de la clave.
- Las posiciones $1, k+1, 2k+1, \dots$ también.

Entonces el texto cifrado puede separarse en **$k$ columnas**, donde cada columna fue cifrada con un único desplazamiento (un César).

Se prueba con $k=1,2,3, \dots$ y se calcula el IC de cada columna

- Texto en lenguaje natural (español/inglés): $
  IC \approx 0.065
  $

- Texto completamente aleatorio:
  $
  IC \approx \frac{1}{26} \approx 0.038
  $

#### Como descifrar?
Entonces hacemos esto:

- Por cada columna:
  - Probar los 26 desplazamientos y ver cuál produce texto que parece inglés
  - usar la formula de chi cuadrado para evaluar si la distribución del decifrado se parece al inglés
  - un chi cuadrado pequeño dice que si se parece al inglés
  - un chi cuadrado grande dice que no se parece

$$
X^2= \sum^{25}_{i=0} \frac{(O_i-E_i)^2}{E_i}
$$

Donde $O_i$ es la frecuencia observada de la letra $i$ en el texto y E_i es la frecuencia esperada de la letra $i$ en en ingles

In [ ]:
def ic(lista):
    
    N = len(lista)
    if N <= 1:
        return 0
    
    sumatoria = 0
    for i in range(26):
        f = lista.count(i)
        sumatoria += f*(f-1)
    
    return sumatoria / (N*(N-1))

def columnas(lista: list, j:int):
    # Separa una lista en j columnas
    out = [[] for _ in range(j)]
    # print(out)
    j_ = 0
    for a in lista:
        # print(out)
        out[j_].append(a)
        j_= j_+1 if j_<j-1 else 0
    return out

def chi2(lista, frecuencias_ingles):
    N = len(lista)
    chi = 0
    for i in range(26):
        O_i = lista.count(i)
        E_i = frecuencias_ingles[i] * N
        if E_i > 0:
            chi += ((O_i - E_i) ** 2) / E_i
    return chi

def descifrar_con_cesar(lista, desplazamiento):
    out = ["" for _ in lista]
    for a in range(len(lista)):
        decifrado = (lista[a]-desplazamiento) %26
        out[a] = decifrado
    return out

def descifrar_vigenere(texto, clave):
    texto_lista = [letra_a_numero(a) for a in texto]
    clave_lista = [letra_a_numero(a) for a in clave]
    while len(texto_lista) > len(clave_lista):
        clave_lista += clave_lista
    while len(texto_lista) != len(clave_lista):
        clave_lista.pop()
    
    decifrado = [(texto_lista[i] - clave_lista[i]) %26 for i in range(len(texto_lista))]
    out = ""
    for a in decifrado:
        out += numero_a_letra(a)
    return out


### 1. Encontrar el periodo j de la clave k
Utilizando el ataque visto en clase, utilizando el indice de coincidencia, encuentre el periodo j de la clave secreta k (busque entre 1 y 10).

In [114]:
message
msg_list = [letra_a_numero(a) for a in message]

In [115]:
ics = [0]
for a in range(1, 10):
    cols = columnas(msg_list, a)
    
    # Lista con los ic de las columnas
    ic_s=[ic(b) for b in cols]
    
    #promedio de ic asumiendo un tamaño a
    sum = 0
    for c in ic_s:
        sum+=c
    ics.append(sum/len(ic_s))
    

In [116]:
display(ics)
j = ics.index(max(ics))
print(f"Tamaño clave: {j}")

[0,
 0.04295012462071955,
 0.04679125269856341,
 0.04288484524808626,
 0.05400005877618931,
 0.04140967391741386,
 0.04756502548373314,
 0.041258809148717414,
 0.06846711656719118,
 0.04274350567100225]

Tamaño clave: 8


La posición 8 es la más cercana a 0.065, por lo que lo mas posible es que $j=len(K) = 8$

### 2. Encontrar clave secreta y texto original
Utilizando el ataque visto en clase, encuentre la clave secreta k y el texto orignal (que esta en ingles).

In [117]:
english_freq = {
    'a': 0.08167, 'b': 0.01492, 'c': 0.02782, 'd': 0.04253,
    'e': 0.12702, 'f': 0.02228, 'g': 0.02015, 'h': 0.06094,
    'i': 0.06966, 'j': 0.00153, 'k': 0.00772, 'l': 0.04025,
    'm': 0.02406, 'n': 0.06749, 'o': 0.07507, 'p': 0.01929,
    'q': 0.00095, 'r': 0.05987, 's': 0.06327, 't': 0.09056,
    'u': 0.02758, 'v': 0.00978, 'w': 0.02360, 'x': 0.00150,
    'y': 0.01974, 'z': 0.00074
}
en_list = [a for a in english_freq.values()]

In [118]:
cols = columnas(msg_list, j)

In [ ]:
chis2 = []
ks=[]
for a in cols:
    mejor_k = None
    mejor_chi = float("inf")

    for k in range(26):
        lista_descifrada = descifrar_con_cesar(a, k)
        chi = chi2(lista_descifrada, en_list)
        
        if chi < mejor_chi:
            mejor_chi = chi
            mejor_k = k
    chis2.append(mejor_chi)
    ks.append(mejor_k)
print(f"Desplazamientos por columna: {ks}")

Posibles desplazamientos por columna: [20, 13, 8, 0, 13, 3, 4, 18]


In [126]:
k = ""
for a in ks:
    k+=numero_a_letra(a)
k

'UNIANDES'

In [ ]:
descifrar_vigenere(message, k)

'ONCEUPONATIMETHEREWASASWEETLITTLEGIRLLOVEDBYEVERYONEWHOMETHERBUTMOSTOFALLBYHERGRANDMOTHERTHEOLDWOMANADOREDHERSOMUCHTHATSHEMADEHERASMALLREDVELVETHOODITSUITEDHERPERFECTLYANDFROMTHATDAYONEVERYONECALLEDHERLITTLEREDRIDINGHOODONEMORNINGHERMOTHERSAIDCOMELITTLEREDRIDINGHOODHERESAPIECEOFCAKEANDABOTTLEOFWINETAKETHEMTOYOURGRANDMOTHERSHESSICKANDWEAKANDTHISWILLDOHERGOODGOBEFOREITGETSTOOHOTANDREMEMBERTOWALKCAREFULLYDONTSTRAYFROMTHEPATHORYOUMIGHTFALLANDBREAKTHEBOTTLEANDWHENYOUARRIVEDONTFORGETTOSAYGOODMORNINGBEFOREYOUPEEKAROUNDHERROOMILLBECAREFULMOTHERSAIDLITTLEREDRIDINGHOODANDSHEPROMISEDWITHASMILETHEGRANDMOTHERLIVEDDEEPINTHEFORESTABOUTHALFALEAGUEFROMTHEVILLAGEJUSTASLITTLEREDRIDINGHOODENTEREDTHEWOODSAWOLFAPPEAREDONTHEPATHSHEDIDNTKNOWWHATAWICKEDCREATUREHEWASANDFELTNOFEARATALL'